In [1]:
import os
import jax

In [3]:
import quicktack
import ticktack

In [4]:
@jax.jit
def production(t, *args):
    """
    This is the production term of the carbon box model. This production function contains a sinusoidal solar term and a super-gaussian event. 

    Parameters:
    -----------
    : t : The time dependence of the production 
    : args : The parameters of the solar amplitude function and the event

    Returns:
    --------
    : DeviceArray : The production at the time t 
    """
    start_time, duration, phase, area = jax.numpy.array(args)
    middle = start_time + duration / 2.
    height = area / duration

    gauss = height * \
        jax.numpy.exp(- ((t - middle) / (1. / 1.93516 * duration)) ** 16.)
    sine = 1.8803862513018528 + 0.18 * 1.8803862513018528 *\
        jax.numpy.sin(2 * jax.numpy.pi / 11 * t +
                      phase * 2 * jax.numpy.pi / 11)

    return (sine + gauss) * 3.747273140033743

In [5]:
time = jax.numpy.linspace(0, 100, 100)
args = (774.86, 0.25, 0.8, 6.44)

In [6]:
test = quicktack.load(f"{os.getcwd()}/data/Guttler15.hd5")

In [7]:
test.equilibrate()

In [10]:
%%timeit
test.run(production, time, test._equilibrium, args)

6.1 ms ± 444 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
model = ticktack.load_presaved_model("Guttler15", production_rate_units="atoms/cm^2/s")
model.compile()

In [31]:
equilibrium = model.equilibrate(production_rate=1.88)

In [35]:
%%timeit
model.run(time, production, equilibrium, args, steady_state_production=1.88)

6.53 ms ± 402 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
